# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
print(df.head(30))
df.info()

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

**Вывод**
В таблице присутствуют пропуски в столбце "общий трудовой стаж", скорее всего из-за того, что человек работал неофициально, без оформления через трудовую книжку и не смог подтвердить свой стаж, что также обьясняет отсутствие данных и по ежемесячному доходу. Для госслужещих эта ошибка, скорее всего носит случайный характер (id=26, например), хотя данные о принадлежности к госслужбе, скорее всего, записаны со слов клиента банка, так как он не предоставил документов с работы. В столбце "образование" данные внесены буквами с разным регистром. В столбце "цель получения кредита" одинаковые по смыслу цели записаны разными названиями.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
total_income_tab = df.dropna()
df['education_lowercase'] = df['education'].str.lower()

income_type_dic = df.groupby('income_type')['total_income'].median()
 
group_income_type = df.groupby(['income_type'])
df['total_income'] = group_income_type['total_income'].apply(lambda x: x.fillna(x.median()))

df['days_employed'] = group_income_type['days_employed'].apply(lambda x: x.fillna(x.median()))



print(len(df[df['days_employed'].isna()]))

print(len(df[df['total_income'].isna()]))


0
0


**Вывод**
Пропущенных значений 10% от всех данных, поэтому избавляться от них нельзя. Заменим пропущенные значения на 0.

### Замена типа данных

Возможно XNA в графе пол это просто пропуски, когда человек не указал пол в графе, при заполнении анкеты.

In [3]:
df['days_employed'] = df['days_employed'].astype('int')

df['total_income'] = pd.to_numeric(df['total_income'])

df.info()
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children               21525 non-null int64
days_employed          21525 non-null int64
dob_years              21525 non-null int64
education              21525 non-null object
education_id           21525 non-null int64
family_status          21525 non-null object
family_status_id       21525 non-null int64
gender                 21525 non-null object
income_type            21525 non-null object
debt                   21525 non-null int64
total_income           21525 non-null float64
purpose                21525 non-null object
education_lowercase    21525 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ MB
       children  days_employed  dob_years education  education_id  \
0             1          -8437         42    высшее             0   
1             1          -4024         36   среднее             1   
2             0          -5623         33 

**Вывод** В столбце days_employed хранятся данные об общем трудовом стаже в днях, поэтому значения должны быть целочисленными. Используем для этого метод astype с аргументом 'int'. Для столбца total_income подойдет тип данных float, для этого используем метод to_numeric.

Аномально большие значения, скорее всего из-за того, что человек в нескольких местах одновременно работал или ошибка ввода данных, часы вместо дней. 

### Обработка дубликатов

In [4]:
print(df.duplicated(['days_employed','dob_years', 'education_id', 'family_status_id', 'total_income']).sum())
df = df.drop_duplicates(['days_employed','dob_years', 'education_id', 'family_status_id', 'total_income']).reset_index(drop=True)
df

1389


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,высшее
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,среднее
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,среднее
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,среднее
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,среднее
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20131,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,среднее
20132,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,среднее
20133,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,среднее
20134,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,среднее


**Вывод** Для поиска дубликатов будем использовать методы duplicated и sum. Для удаления дубликатов будем использовать метод drop_duplicates. Так как данные у нас могут совпадать естесственным образом (возраст и стаж не уникальные параметры для людей), то будем искать дубликаты по совпадению в 4 столбцах 'days_employed','dob_years', 'education_id', 'family_status_id', 'total_income' (такое совпадение точно не может быть случайным). Таким образом мы избавимся от 1731 строки с дубликатами.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()
purpose_unique = df['purpose'].unique()
print(purpose_unique)
lemmas = []
for i in purpose_unique:
    lemma = m.lemmatize(i)
    lemmas.append(lemma)
print(lemmas)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']
[['покупка', ' ', 'жилье', '\

**Вывод** Для лемматизации данных в столбце "цель получения кредита" использовалась библиотека pymystem3. Создадим список из уникальных значений целей кридита и применим к нему цикл, который лемматезирует строки из этого списка. На основание полученных данных можно сделать выводы, что все цели кредита можно свести к 4 категориям: приобретение недвижимости, приобретение автомобиля, образование и свадьба.

### Категоризация данных

In [6]:
def purpose_cat(row):
    lemmas_row = m.lemmatize(row)
    if 'автомобиль' in lemmas_row:
        return "приобретение автомобиля"   
    if 'образование' in lemmas_row:
        return "образование"  
    if 'свадьба' in lemmas_row:
        return "свадьба"
    if 'недвижимость' or 'жилье' in lemmas_row:
        return  "операции с недвижимостью"
        
df['purpose_cat'] = df['purpose'].apply(purpose_cat)
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_cat
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,высшее,операции с недвижимостью
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,среднее,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,среднее,операции с недвижимостью
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,среднее,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,среднее,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,высшее,операции с недвижимостью
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,высшее,операции с недвижимостью
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,среднее,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,высшее,свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,среднее,операции с недвижимостью


**Вывод** Введем 4 категории для целей получения кредита: "операции с  недвижимостью" (сюда же войдут схожие по логике цели со словами "недвижимость" и "жилье"), "приобретение автомобиля" (не важно подержанного или нового), "образование", "свадьба".  Лемматезируем значения в строках столбца "purpose" и применим функцию с циклом if.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
df['children'].value_counts()
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 0)
df['children'].value_counts()
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = df.groupby('children')['debt'].count()
debt_from_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children.sort_values('result_children', ascending = False)

,count_children,sum_children,result_children
children,,,
2,1907,181,0.094913
1,4557,425,0.093263
4,36,3,0.083333
3,305,23,0.075410
0,13323,994,0.074608
5,8,0,0.000000


**Вывод** Для начала посмотрим уникальные значения столбца "дети". В нем заметны странности в виде отрицательного количества детей и 20 детей у слишком большого количества клиентов банка. Возможно минус внесли по-ошибке. Как возникло значение 20 неизвестно, не будем учитывать данные этих строк.
Для семей с разным количеством детей посчитаем отношение количества этих семей к количеству просроченных кредитов и результат выведем в таблицу. Из нее видно, что семьи без детей реже имеют задолженности по выплате кредита. Больше всего задолженностей у семей с 1 или 2 детьми. Но выборка по семьям с 3, 4 детьми не достаточная, чтобы делать глобальные выводы.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
df['family_status'].value_counts()
debt_from_family_status = pd.DataFrame()
debt_from_family_status['sum_family_status'] = df.groupby('family_status')['debt'].sum()
debt_from_family_status['count_family_status'] = df.groupby('family_status')['debt'].count()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] 
print(debt_from_family_status.sort_values('result_family_status', ascending = False))
print('___________________________________________________')
data_pivot = df.pivot_table(index=['family_status'], columns='education_id', values='debt', aggfunc='sum')
print(data_pivot)

                       sum_family_status  count_family_status  \
family_status                                                   
Не женат / не замужем                265                 2686   
гражданский брак                     362                 3928   
женат / замужем                      857                11435   
в разводе                             82                 1164   
вдовец / вдова                        60                  923   

                       result_family_status  
family_status                                
Не женат / не замужем              0.098660  
гражданский брак                   0.092159  
женат / замужем                    0.074945  
в разводе                          0.070447  
вдовец / вдова                     0.065005  
___________________________________________________
education_id               0      1     2     3    4
family_status                                       
Не женат / не замужем   44.0  200.0  18.0   3.0  0.0
в разводе  

**Вывод** Завистимость между семейным положением и возвратом кредита в срок есть. Чаще всего задерживают выплату кредита Не женатые/не замужние клиенты банка. По разведенным и вдовцам данных слишком мало, для выводов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
total_income_tab.mean()
def total_income_category(total_income):
    if total_income >= 167422:
        return 1
    return 0
total_income_tab['total_income_id'] = total_income_tab['total_income'].apply(total_income_category)
print(total_income_tab.head(15))

debt_from_total_income = pd.DataFrame()
debt_from_total_income['sum'] = total_income_tab.groupby('total_income_id')['debt'].sum()
debt_from_total_income['count'] = total_income_tab.groupby('total_income_id')['debt'].count()
debt_from_total_income['conversion'] = debt_from_total_income['sum'] / debt_from_total_income['count']
debt_from_total_income.sort_values('conversion', ascending = False)


    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sum,count,conversion
total_income_id,,,
0,1003,11892,0.084342
1,568,7459,0.076150


**Вывод** Для определения зависимости между уровнем дохода и возвратом кредита в срок посчитаем средний уровень дохода, методом mean, примененным к столбцу с общим доходом, после удаления строк с отсутствующим значением. Поделим клиентов на 2 группы: с доходом ниже среднего (id=0) и выше среднего (id=1). Посчитаем конверсию и окажется, что люди с меньшим доходом чаще задерживают выплаты по кредитам.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
debt_from_purpose_cat = pd.DataFrame()
debt_from_purpose_cat['sum_purpose_cat'] = df.groupby('purpose_cat')['debt'].sum()
debt_from_purpose_cat['count_purpose_cat'] = df.groupby('purpose_cat')['debt'].count()
debt_from_purpose_cat['result_purpose_cat'] = debt_from_purpose_cat['sum_purpose_cat'] / debt_from_purpose_cat['count_purpose_cat'] 
debt_from_purpose_cat.sort_values('result_purpose_cat', ascending = False)


,sum_purpose_cat,count_purpose_cat,result_purpose_cat
purpose_cat,,,
приобретение автомобиля,383,4049,0.094591
образование,338,3721,0.090836
свадьба,171,2210,0.077376
операции с недвижимостью,734,10156,0.072273


**Вывод** Цель кредита влияет на его возврат в срок. Клиенты, берущие кредиты на приобретение автомобиля и образование чаще задерживают выплаты.

## Шаг 4. Общий вывод



На срок возврата кредита влияют разные факторы. Клиенты, состоящие в браке реже имеют задолжности по возвращению кредита в срок, в отличие от клиентов не состоящих в браке. Семьи без детей лучше отдают кредит в срок, чем семьи с 1, 2 детьми, по семьям с большим количеством детей недостаточно данных.